## 네이버 금융에서 Daily 환율 정보 가져오기

In [1]:
import pandas as pd
import requests, csv
from bs4 import BeautifulSoup, NavigableString, Tag
from tqdm import tqdm

In [2]:
URL = 'https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_USDKRW&page=1'

response = requests.get(URL)
soups = BeautifulSoup(response.text, 'html.parser')

In [3]:
currency_df = pd.DataFrame()
for i in tqdm(range(1, 95)):
    URL = f'https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_USDKRW&page={i}'
    response = requests.get(URL)
    soups = BeautifulSoup(response.text, 'html.parser')
    
    for currency_part in soups.select('body > div > table > tbody > tr'):
        date = pd.to_datetime(currency_part.select_one('.date').text)
        currency = float(currency_part.select_one('.num').text.replace(',', ''))

        df = pd.DataFrame({'Date':[date], 'Currency':[currency]})
        currency_df = pd.concat([currency_df, df], axis = 0)

100%|███████████████████████████████████████████| 94/94 [00:09<00:00,  9.73it/s]


In [4]:
currency_df = currency_df.sort_values('Date').reset_index(drop = True)
print(currency_df.shape)
currency_df.head()

(940, 2)


,Date,Currency
0,2018-05-29,1080.0
1,2018-05-30,1080.0
2,2018-05-31,1078.0
3,2018-06-01,1075.5
4,2018-06-04,1070.0


In [5]:
currency_df.to_excel('/Users/paul/Desktop/currency_data.xlsx')